<h2> Generating the Stimuli file</h2>
    
This notebook is a modified version of prior work to generate the stimuli files used in anagram experiments.




#### The word bank we are using is from Word Net:
    George A. Miller (1995). WordNet: A Lexical Database for English.
    Communications of the ACM Vol. 38, No. 11: 39-41.
    Christiane Fellbaum (1998, ed.) WordNet: An Electronic Lexical Database. Cambridge, MA: MIT Press.
    WordNet: An Electronic Lexical Database

#### The frequency information is from the word freq: 
    Robyn Speer. (2022). rspeer/wordfreq: v3.0 (v3.0.2). Zenodo. https://doi.org/10.5281/zenodo.7199437


In [1]:
%load_ext autoreload
%reload_ext autoreload

import csv
import json
import random
import pandas as pd
from collections import Counter

import nltk
import numpy as np
import itertools
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
#import spacy 
from nltk.stem import WordNetLemmatizer

from anagram_utils import (
    mk_dict_from_wordnet_for_length,
    remove_from_dict,
    get_word_frequencies,
    sort_words_by_frequency,
    get_top_n_words,
    shuffle_list,
    reformat_sorted_wordlist,
    check_for_word,
    check_for_doubles,
    check_for_same,
    find_valid_words
)
nltk.download("wordnet") 
#import spacy.cli
#spacy.cli.download('en_core_web_sm')
#nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lyndefolsom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
word_dict = {}
wordlengths = [4,5,6] # set the lengths outside of the loop bc we use them later too 
for wordlength in wordlengths:
    word_dict[wordlength] = mk_dict_from_wordnet_for_length(wordlength)
    print(f'found {len(word_dict[wordlength])} words of length {wordlength}')

# define our curse words or other words to remove
curse_words = ["shit", "piss", "fuck", "cunt", "cocksucker", "motherfucker", "tits"] #rip george carlin
other_words_to_remove = ["jesus", "george", "john", 'james', 'york', 'david', 'google', 'robert', 'thomas','kill','trump', 'stupid', 'centre' ]
# use the remove from dict func to take those ones out
for wordlength in word_dict:
    word_dict[wordlength] = remove_from_dict(word_dict[wordlength], curse_words)
    print(f'found after removing curse words {len(word_dict[wordlength])} words of length {wordlength}')

for wordlength in word_dict:
    word_dict[wordlength] = remove_from_dict(word_dict[wordlength], other_words_to_remove)
    print(f'found after removing other words {len(word_dict[wordlength])}words of length {wordlength}')



found 2310 words of length 4
found 4095 words of length 5
found 6258 words of length 6
found after removing curse words 2307 words of length 4
found after removing curse words 4095 words of length 5
found after removing curse words 6258 words of length 6
found after removing other words 2304words of length 4
found after removing other words 4091words of length 5
found after removing other words 6252words of length 6


In [12]:
# making a function that will see if a word is in the dictionary, if it is, print "___ is in the dictionary" if not, print "____ is not in the dictionary", then check if it is in the word list and say "___ is in the word list" or "___ is not in the word list" 
# def check_word(word, word_dict):
#     if word in word_dict:
#         print(f'{word} is in the dictionary')
#     else:
#         print(f'{word} is not in the dictionary')
#     if word in full_list:
#         print(f'{word} is in the word list')
#     else:
#         print(f'{word} is not in the word list')
#     return


In [14]:
word_frequencies = get_word_frequencies(word_dict) # we get the word frequencies
sorted_wordlist = sort_words_by_frequency(word_frequencies) # we sort the dictionary by frequency
sorted_wordlist = reformat_sorted_wordlist(sorted_wordlist) # we reformat the dictionary for the get_top_n_words function
# make your subset list of words for the anagrams
number_of_anagrams_per_word_length = 200
full_list = get_top_n_words(sorted_wordlist, number_of_anagrams_per_word_length)
# kewl so lets make a list of those words but shuffled letters ie our anagrams
shuffled_list = shuffle_list(full_list) # we take the list and make a new list of each word's letters shuffled (anagram)
# and for the next few things it's gonna help to also have a df of these two lits named root and shuffled
cat_full_list = pd.DataFrame({
    "root" : full_list,
    "shuffled" : shuffled_list,
    # id collumn which is a string of the index + 1 so that it's 3 digits long
    "id" : [str(i+1).zfill(3) for i in range(len(full_list))]    
})
cat_full_list= check_for_doubles(cat_full_list) # we check for any words that are the same in the root and shuffled columns 
cat_full_list= check_for_same(cat_full_list) # we check for any duplicates in the shuffled coll and reshuffle them
## need to make a function that checks that words that are "shuffled" do not exist in the dictionary, then shuffle if they are
cat_full_list = check_for_word(cat_full_list, word_dict)



In [15]:
from nltk.corpus import wordnet as wn

def check_word_in_wordnet(word):
    if wn.synsets(word):
        print(f'{word} is in the WordNet corpus')
    else:
        print(f'{word} is not in the WordNet corpus')

check_word_in_wordnet('except')




except is in the WordNet corpus
[Lemma('demur.v.01.except'), Lemma('exclude.v.01.except')]
[]


In [16]:
def is_english_word(word):
    """
    Check if a word exists in WordNet's dictionary
    
    Args:
        word (str): The word to check
        
    Returns:
        bool: True if the word exists in WordNet, False otherwise
    """
    # Convert to lowercase for consistency
    word = word.lower().strip()
    
    # Check if the word exists in WordNet
    return len(wn.synsets(word)) > 0

def get_all_permutations(word):
    return [''.join(p) for p in itertools.permutations(word)]

def get_all_permissible_words(input_word):
    perms = get_all_permutations(input_word)
    return [word for word in perms if is_english_word(word)]

get_all_permissible_words('nets')

['nets', 'nest', 'tens', 'sent']

## Make a solution key

Many anagrams have more than one response that is a valid solution. 

In [17]:

# # then check agasint the word_dict and remove them from the permutations list if they are in the dictionary
# valid_words_gram = [find_valid_words(word, word_dict) for word in shuffled_list] # we find the valid words for each anagram in the shuffled list
# valid_words = [list(set(words)) for words in valid_words_gram] # we remove any duplicates in the valid words list and set to just the one list


# # from a shuffled word list, we if a word is not in the dictionary then we add it to a permutations list
# def find_invalid_perms(word, word_dict):
#     permutations = ["".join(i) for i in itertools.permutations(word)]
#     invalid_perms = []
#     for candidate in permutations:
#         if candidate not in word_dict:
#             invalid_perms.append(candidate)
#     return invalid_perms

# invalid_perms = [find_invalid_perms(word, word_dict) for word in shuffled_list]

# okay now lets use the get_all_permissible_words function to get the valid words for each anagram
valid_words_gram = [get_all_permissible_words(word) for word in shuffled_list]
valid_words_gram = [list(set(words)) for words in valid_words_gram]

# # remove the valid_words from the permutations list
# print(len(invalid_perms))
# for word in valid_words:
#     if word in invalid_perms:
#         invalid_perms.remove(word)

# # remove duplicates in eachline of the invalid perms list
# invalid_perms = [list(set(words)) for words in invalid_perms]




In [18]:
# valid_words_gram = [find_valid_words(word, word_dict) for word in shuffled_list] # we find the valid words for each anagram in the shuffled list
# valid_words = [list(set(words)) for words in valid_words_gram] # we remove any duplicates in the valid words list and set to just the one list
# cat_full_list['valid_words'] = valid_words # we add the valid words to the df
# cat_full_list=check_for_word(cat_full_list, word_dict) # we check for any words that are the same in the root and shuffled columns

# lets do this again but witht he valid_words_with_newfunc
valid_words_gram = [get_all_permissible_words(word) for word in shuffled_list] # we find the valid words for each anagram in the shuffled list
valid_words_gram = [list(set(words)) for words in valid_words_gram] # we remove any duplicates in the valid words list and set to just the one list
cat_full_list['valid_words'] = valid_words_gram # we add the valid words to the df




In [19]:
# function to use the root in the df to assign it's string length in a new column of the same def
def get_length(row):
    return len(row['root']) 

# apply the function to the df
cat_full_list['length'] = cat_full_list.apply(get_length, axis=1)

# function to first group by the length in a DF then randomly assign to a group a through d 
def group_by_length(df):
    #
    if len(df) % 4 != 0:
        raise "Error in distributing between 4 groups, please revise counts"
    # group by the length of the word
    grouped = df.groupby('length')
    # make a list of the groups
    groups = [group for name, group in grouped]
    # make a list of the group names
    group_names = ['SetA', 'SetB', 'SetC', 'SetD']
    # calculate number of words per group
    num_words_per_group = len(df) // len(group_names)
    # make a dictionary of the group names and the groups
    group_dict = dict(zip(group_names, groups))
    # make a list of the group names for each word in the df don't exceed the num words per group
    group_list = [] # make an empty list to append to
    # loop through the group names until the number of words in the group is equal to the number of words per group
    for group_name in group_names:
        for i in range(num_words_per_group):
            group_list.append(group_name)
    # assign the group list to the df
    df['Set'] = group_list
    return df

# apply the function to the df
cat_full_list = group_by_length(cat_full_list)

# check that each group in the df has the same number of words
grouped = cat_full_list.groupby('Set')
for name, group in grouped:
    print(f'{name} has {len(group)} words')

#### troublesome function ***** now we create 4 unique set runs for each set and call that col setrun and it uses the val of set and adds it's number
### so that we have 4 unique runs for each set, this would then mean our final js stim  would have 16 unique runs, 4 for each set, and a file that ought to be len(id) * 16

def set_run(df):
    # group by set
    grouped = df.groupby('Set')
    # for each group, shuffle and assign it to a set run A1, A2, A3, A4, B1, B2, B3, B4, etc.
    set_runs_list = []
    # loop through the groups four times, each time shuffle the order and then append taht list to the set runs
    for name, group in grouped:
        for i in range(4):
            # take the id and shuffle the order (but not the contents, just the order) and append to the set runs list
            set_runs_list.append(group['id'].sample(frac=1).tolist())
    # make a new column in the df and assign the set runs list to it
    df['set_run'] = set_runs_list
    return df
#save the cat_full_list to a csv
cat_full_list.to_csv('cat_full_list.csv', index=False)


SetA has 150 words
SetB has 150 words
SetC has 150 words
SetD has 150 words


In [20]:
full_list = pd.read_csv('cat_full_list.csv') #def gonna need to be optimized

target_sets = full_list['Set'].unique()
nlists = 4
rand_full_list = None 

for target_set in target_sets:
    for list_num in range(1, nlists+1):
        set_list_name = f'{target_set}{list_num}'
        target_list = full_list[full_list['Set'] == target_set]
        # randoly shuffle order of target list
        target_list = target_list.sample(frac=1).reset_index(drop=True)
        target_list['SetRun'] = set_list_name
        if rand_full_list is None:
            rand_full_list = target_list
        else:
            rand_full_list = pd.concat([rand_full_list, target_list])


def format_js_stimuli(df):
    js_stimuli = []
    for idx, row in df.iterrows():
        js_entry = {
                "id": str(row['id']),
                "type": str(row['length']),
                "anagram": row['shuffled'],
                "correct": row['root'],
                "valid": row['valid_words'],
                "set": row['Set'],
                "setRun": row['SetRun'],
            }
        js_stimuli.append(js_entry) 
    return js_stimuli

rand_full_list_json = format_js_stimuli(rand_full_list)
print(len(rand_full_list_json))
print(rand_full_list_json[0])

with open('rand_full_list.json', 'w') as f:
    json.dump(rand_full_list_json, f, indent=4)

with open('rand_full_list.json', 'r') as f:
    rand_full_list_json_lines = f.readlines()

first_line = "let trial_objects = [\n" 
last_line = "];"
rand_full_list_json_lines[0] = first_line
rand_full_list_json_lines[-1] = last_line


with open('stimuli.js', 'w') as f:
    f.writelines(rand_full_list_json_lines)


2400
{'id': '90', 'type': '6', 'anagram': 'rttmea', 'correct': 'matter', 'valid': "['matter']", 'set': 'SetA', 'setRun': 'SetA1'}
